In [1]:
from paths import DATA_DIR, CKPT_FOLDER, PROJECT_FOLDER

import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import anndata
import scanpy as sc
import sklearn
import scvelo as scv
from pathlib import Path
import seaborn as sns
import pertpy as pt

import anndata
import pandas as pd

from IPython.display import display
from torchdyn.core import NeuralODE

from scCFM.datamodules.time_sc_datamodule import TrajectoryDataModule
from scCFM.models.cfm.components.mlp import MLP
from scCFM.models.cfm.cfm_module import CFMLitModule

from scCFM.models.base.vae import VAE
from scCFM.models.base.geometric_vae import GeometricNBVAE
from scCFM.models.base.geodesic_ae import GeodesicAE

from scCFM.datamodules.sc_datamodule import scDataModule
from scCFM.models.cfm.components.eval.distribution_distances import compute_distribution_distances

from notebooks.utils import decode_trajectory_single_step, standardize, compute_prdc

from tqdm import tqdm
import time
import pandas as pd
import warnings

ryp2 is not installed. Install with pip install rpy2 to run tools with R support.

To use sccoda or tasccoda please install ete3 with pip install ete3

Initialize datamodule

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

datamodule_config={'path': PROJECT_FOLDER / 'data/eb/processed/eb_phate.h5ad', 
                    'x_layer': 'X_norm', 
                    'cond_keys': ['experimental_time', 'leiden'],
                    'use_pca': False, 
                    'n_dimensions': None, 
                    'train_val_test_split': [1], 
                    'batch_size': 256, 
                    'num_workers': 2}

# Initialize datamodule
datamodule = scDataModule(**datamodule_config)

Initialize configurations

In [3]:
vae_kwargs={'in_dim': datamodule.in_dim,
               'n_epochs_anneal_kl': 1000, 
               'kl_weight': None, 
               'likelihood': 'nb', 
               'dropout': False, 
               'learning_rate': 0.001, 
               'dropout_p': False, 
               'model_library_size': True, 
               'batch_norm': True, 
               'kl_warmup_fraction': 0.1, 
               'hidden_dims': [256, 10]}
        
geometric_kwargs={'compute_metrics_every': 1, 
                   'use_c': False, 
                   'trainable_c': False,
                   'l2': True, 
                   'eta_interp': 0, 
                   'interpolate_z': False, 
                   'start_jac_after': 0, 
                   'fl_weight': 0.1,
                   'detach_theta': True}

geodesic_kwargs={"in_dim": datamodule.in_dim,
                  "hidden_dims": [256, 10],
                  "batch_norm": True,
                  "dropout": False, 
                  "dropout_p": False,
                  "likelihood": "nb",
                  "learning_rate": 0.001}

In [4]:
# Suppress warnings
warnings.filterwarnings("ignore")

# Define batch sizes to test
batch_sizes = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
num_examples = 10  # Number of examples to run for each batch size

## Without backpropagation

In [5]:
# Initialize empty lists to store results
results = {"model": [],
           "batch_size": [], 
           "runtime": []
          }

# Iterate over batch sizes
for batch_size in batch_sizes:
    # Repeat for each batch size
    for _ in range(num_examples):
        # Set batch size in datamodule
        datamodule_config["batch_size"] = batch_size
        datamodule = scDataModule(**datamodule_config)
        batch = next(iter(datamodule.train_dataloader()))
        batch["X"] = batch["X"].cuda()
        
        # Initialize vae and geometric vae
        vae = VAE(**vae_kwargs).to(device)
        geometric_vae = GeometricNBVAE(**geometric_kwargs, vae_kwargs=vae_kwargs).to(device)
        geodesic_ae = GeodesicAE(**geodesic_kwargs).to(device)

        # Time forward step for geodesic_ae
        start_time = time.time()
        geodesic_ae.step(batch, "train")
        end_time = time.time()
        geodesic_ae_runtime = end_time - start_time
        results["model"].append("geodesic")
        results["batch_size"].append(batch_size)
        results["runtime"].append(geodesic_ae_runtime)

        # Time forward step for geometric_vae
        start_time = time.time()
        geometric_vae.step(batch, "train")
        end_time = time.time()
        geometric_ae_runtime = end_time - start_time
        results["model"].append("geometric")
        results["batch_size"].append(batch_size)
        results["runtime"].append(geometric_ae_runtime)

        # Time forward step for vae
        start_time = time.time()
        vae.step(batch, "train")
        end_time = time.time()
        vae_runtime = end_time - start_time
        results["model"].append("vae")
        results["batch_size"].append(batch_size)
        results["runtime"].append(vae_runtime)

        del batch 
        del vae
        del geometric_vae
        del geodesic_ae
        
# Create dataframe
df = pd.DataFrame(results)

In [6]:
df.groupby(["model","batch_size"]).mean()

runtime
model     batch_size           
geodesic  4            1.831309
          8            0.118640
          16           0.094533
          32           0.114758
          64           0.098721
          128          0.116311
          256          0.217593
          512          0.512575
          1024         1.521877
          2048         7.011410
          4096        67.397131
geometric 4            0.205671
          8            0.008975
          16           0.007100
          32           0.004913
          64           0.004666
          128          0.009441
          256          0.009466
          512          0.015232
          1024         0.009434
          2048         0.014179
          4096         0.073186
vae       4            0.001629
          8            0.001670
          16           0.001647
          32           0.001715
          64           0.001571
          128          0.001994
          256          0.001901
          512          0.001564
          1024         0.001590
          2048         0.001860
          4096         0.003826

In [7]:
df.groupby(["model","batch_size"]).std() / np.sqrt(10)

runtime
model     batch_size          
geodesic  4           1.697728
          8           0.011796
          16          0.000549
          32          0.019420
          64          0.002376
          128         0.006646
          256         0.017366
          512         0.020542
          1024        0.013141
          2048        0.019636
          4096        1.738363
geometric 4           0.200931
          8           0.004282
          16          0.002446
          32          0.000250
          64          0.000160
          128         0.003921
          256         0.001858
          512         0.004252
          1024        0.000697
          2048        0.000398
          4096        0.025468
vae       4           0.000102
          8           0.000058
          16          0.000030
          32          0.000052
          64          0.000048
          128         0.000066
          256         0.000142
          512         0.000072
          1024        0.000029
          2048        0.000014
          4096        0.000205

## With backpropagation

In [8]:
import warnings
import pandas as pd
import time

# Suppress warnings
warnings.filterwarnings("ignore")

# Initialize empty lists to store results
results_backprop = {"model": [],
                    "batch_size": [],
                    "runtime": []}


# Iterate over batch sizes
for batch_size in batch_sizes:
    # Repeat for each batch size
    for _ in range(num_examples):
        # Set batch size in datamodule
        datamodule_config["batch_size"] = batch_size
        datamodule = scDataModule(**datamodule_config)
        batch = next(iter(datamodule.train_dataloader()))
        batch["X"] = batch["X"].cuda()
        
        # Initialize vae and geometric vae
        vae = VAE(**vae_kwargs).to(device)
        geometric_vae = GeometricNBVAE(**geometric_kwargs, vae_kwargs=vae_kwargs).to(device)
        geodesic_ae = GeodesicAE(**geodesic_kwargs).to(device)

        # Initialize optimizers for each model
        vae_optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)
        geometric_vae_optimizer = torch.optim.Adam(geometric_vae.parameters(), lr=0.001)
        geodesic_ae_optimizer = torch.optim.Adam(geodesic_ae.parameters(), lr=0.001)

        # Time forward step and backward step for geodesic_ae
        start_time = time.time()
        loss = geodesic_ae.step(batch, "train")
        geodesic_ae_optimizer.zero_grad()  # Clear gradients before backward pass
        loss.backward()
        geodesic_ae_optimizer.step()  # Update parameters based on gradients
        end_time = time.time()
        geodesic_ae_runtime = end_time - start_time
        results_backprop["model"].append("geodesic")
        results_backprop["batch_size"].append(batch_size)
        results_backprop["runtime"].append(geodesic_ae_runtime)

        # Time forward step and backward step for geometric_vae
        start_time = time.time()
        loss = geometric_vae.step(batch, "train")
        geometric_vae_optimizer.zero_grad()  # Clear gradients before backward pass
        loss.backward()
        geometric_vae_optimizer.step()  # Update parameters based on gradients
        end_time = time.time()
        geometric_ae_runtime = end_time - start_time
        results_backprop["model"].append("geometric")
        results_backprop["batch_size"].append(batch_size)
        results_backprop["runtime"].append(geometric_ae_runtime)

        # Time forward step and backward step for vae
        start_time = time.time()
        loss = vae.step(batch, "train")
        vae_optimizer.zero_grad()  # Clear gradients before backward pass
        loss.backward()
        vae_optimizer.step()  # Update parameters based on gradients
        end_time = time.time()
        vae_runtime = end_time - start_time
        results_backprop["model"].append("vae")
        results_backprop["batch_size"].append(batch_size)
        results_backprop["runtime"].append(vae_runtime)

        del batch 
        del vae
        del geometric_vae
        del geodesic_ae
        
# Create dataframe
df_backprop = pd.DataFrame(results_backprop)


In [9]:
df_backprop.groupby(["model","batch_size"]).mean()

runtime
model     batch_size           
geodesic  4            0.122397
          8            0.106280
          16           0.097790
          32           0.096211
          64           0.117600
          128          0.220128
          256          0.272736
          512          0.479076
          1024         1.608938
          2048         8.181600
          4096        60.297533
geometric 4            0.031639
          8            0.007633
          16           0.006799
          32           0.007284
          64           0.009216
          128          0.007537
          256          0.008914
          512          0.034436
          1024         0.027587
          2048         0.029567
          4096         0.055320
vae       4            0.003678
          8            0.003419
          16           0.003289
          32           0.003456
          64           0.003905
          128          0.003887
          256          0.004196
          512          0.007819
          1024         0.012189
          2048         0.025259
          4096         0.046026

In [10]:
df_backprop.groupby(["model","batch_size"]).std() / np.sqrt(10)

runtime
model     batch_size          
geodesic  4           0.017907
          8           0.007079
          16          0.003151
          32          0.001001
          64          0.008683
          128         0.020137
          256         0.022164
          512         0.011237
          1024        0.081450
          2048        0.310014
          4096        0.823871
geometric 4           0.024145
          8           0.000685
          16          0.000156
          32          0.000282
          64          0.001314
          128         0.000180
          256         0.000649
          512         0.003870
          1024        0.002677
          2048        0.001934
          4096        0.005391
vae       4           0.000106
          8           0.000097
          16          0.000090
          32          0.000129
          64          0.000298
          128         0.000195
          256         0.000048
          512         0.001349
          1024        0.000381
          2048        0.000351
          4096        0.000943

In [11]:
batch

NameError: name 'batch' is not defined